In [ ]:
# to be completed
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
%cd /content/drive/'My Drive'/'IEMOCAP_full_release_withoutVideos_sentenceOnly'/'IEMOCAP_full_release'

/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release


***UNZIP THE IMOCAP DATASETS***

In [ ]:
#!pip install transformers
#!pip install tensorboardx
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.6 MB/s eta 0:00:00


*PREPROCESIING THE IMPCAP AUDIO FILES* : THANKS TO ***https://github.com/MITESHPUTHRANNEU/Speech-Emotion-Analyzer/blob/master/final_results_gender_test.ipynb  and https://github.com/david-yoon/multimodal-speech-emotion/blob/master/preprocessing/IEMOCAP_01_wav_to_feature.ipynb

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

**read the all the files in a list at sentence level**

In [ ]:
import os
import chardet

def file_search(dirname, ret, list_avoid_dir=[]):
    filenames = os.listdir(dirname)

    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)

        else:
            ret.append( full_filename )

***below code is:***
https://github.com/Escanor1996/Speech-Emotion-Recognition-SER-/blob/master/SER.ipynb

***Paper:*** Attention Based Fully Convolutional Network for Speech Emotion Recognition


In [ ]:
def audio2spectrogram(filepath):
    #fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    #print('samplerate',samplerate)
    _, spectrogram = log_specgram(test_sound, samplerate)
    #print(spectrogram.shape)
    #print(type(spectrogram))
    #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    return spectrogram

def audio2wave(filepath):
    fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    plt.plot(test_sound)

In [ ]:
def log_specgram(audio, sample_rate, window_size=40,
                 step_size=20, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    #print('noverlap',noverlap)
    #print('nperseg',nperseg)
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [ ]:
N_CHANNELS = 3
def get_3d_spec(Sxx_in, moments=None):
    if moments is not None:
        (base_mean, base_std, delta_mean, delta_std,
             delta2_mean, delta2_std) = moments
    else:
        base_mean, delta_mean, delta2_mean = (0, 0, 0)
        base_std, delta_std, delta2_std = (1, 1, 1)
    h, w = Sxx_in.shape
    right1 = np.concatenate([Sxx_in[:, 0].reshape((h, -1)), Sxx_in], axis=1)[:, :-1]
    delta = (Sxx_in - right1)[:, 1:]
    delta_pad = delta[:, 0].reshape((h, -1))
    delta = np.concatenate([delta_pad, delta], axis=1)
    right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
    delta2 = (delta - right2)[:, 1:]
    delta2_pad = delta2[:, 0].reshape((h, -1))
    delta2 = np.concatenate([delta2_pad, delta2], axis=1)
    base = (Sxx_in - base_mean) / base_std
    delta = (delta - delta_mean) / delta_std
    delta2 = (delta2 - delta2_mean) / delta2_std
    stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
    return np.concatenate(stacked, axis=2)

Spectrogram: one axis represents the time(X-axis), the second axis represents frequencies(Y-axis) and the colors represent magnitude (amplitude) of the observed frequency at a particular time

***read the processed transcription file to collect the labels***

In [ ]:
def get_labels(annot_file, file_name):

    f = open(annot_file, 'r').read()
    f = f.split('\n')
    f = f[2:]

    for data in f:

        if len(data) > 0:
            if data[0] == '[':
                data2 = data.split('\t')

                if data2[1] == file_name:
                    emo = data2[2]
                    vad = data2[3][1:-1].split(', ')
                    return emo, [float(x) for x in vad]

    raise ValueError('Label not found')

def get_mocap_rot(path):

    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split(' ')
    xyz = f[1].split(' ')
    f = f[2:]

    data_list = []

    for data in f:
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        dic = {'frame': data2[0], 'time': data2[1],
               'markers': np.array(data2[2:]).astype(float)}
        data_list.append(dic)

    return header, xyz, data_list

def get_wav(path):
    x, sr = librosa.load(path, sr=16000)
    return x, sr

def get_ph_fa(path):
    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split()
    f = f[1:-2]
    data_list = []

    for data in f:
        data2 = data.split()
        dic = {'SFrm':np.array(data2[0]).astype(int),
               'EFrm':np.array(data2[1]).astype(int),
               'SegAScr':np.array(data2[2]).astype(int),
               'Phone':data2[3]}
        data_list.append(dic)

    return header, data_list

def frame_to_s(fr):
    return (fr+2)*10/1000

In [ ]:
import pandas as pd
#'/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release/'
#df=pd.read_excel('/content/drive/My Drive/Imocap_text/processed_tran.xlsx')

root_path = '/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly'

df = pd.read_csv(os.path.join(root_path, 'iemocap.csv'))
sessions = [1, 2, 3, 4, 5]
df = df[df['session'].isin(sessions)]

# Remove unwanted emotions and empty values
unwanted_emotions = ['xxx', '', 'oth', 'dis', 'sur', 'fea']
df = df[~df['emotion'].isin(unwanted_emotions)]

# Calculate annotator difference
df['annotator_difference'] = df['n_annotators'] - df['agreement']

# Filter by annotator difference
df = df[df['annotator_difference'] <= 1]

# Replace 'exc' emotion with 'hap'
df.loc[df['emotion'] == 'exc', 'emotion'] = 'hap'


emotions_count_before = df['emotion'].value_counts()
print("Emotions count before filtering:")
print(emotions_count_before)

# Group by emotion and select first 550 rows of each group
df = df.groupby('emotion').head(550)

# Count the occurrences of each emotion after filtering
emotions_count_after = df['emotion'].value_counts()
print("\nEmotions count after filtering:")
print(emotions_count_after)

# Display the first 5 rows
display(df)


Emotions count before filtering:
fru    1692
neu    1551
hap    1471
ang    1077
sad    1000
Name: emotion, dtype: int64

Emotions count after filtering:
neu    550
fru    550
ang    550
hap    550
sad    550
Name: emotion, dtype: int64


,Unnamed: 0,session,method,gender,n_annotators,agreement,emotion,valence,activation,dominance,wav_path,MOCAP_rotated_path,MOCAP_head_path,MOCAP_hand_path,FA_ph_path,FA_st_path,FA_sy_path,FA_wd_path,annotator_difference
0,0,1,script,F,3,3,neu,2.5,2.0,2.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,0
1,1,1,script,F,3,2,fru,2.5,2.0,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
4,4,1,script,F,3,2,neu,2.0,3.0,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
6,6,1,script,F,3,2,ang,2.0,3.5,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
7,7,1,script,F,3,2,ang,2.0,3.5,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,5350,3,impro,M,4,3,ang,2.0,3.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5493,5493,3,script,M,3,3,ang,2.0,4.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,0
5494,5494,3,script,M,3,2,ang,3.0,2.5,3.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5497,5497,3,script,M,3,2,ang,2.0,4.0,3.5,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_.

In [ ]:
list_files = df['wav_path'].tolist()
labels = df['emotion'].tolist()
#root_path = '/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release/'
#for x in range(5):
 #   sess_name = 'Session' + str(x+1)
  #  path = root_path + sess_name + '/sentences/wav'
   # file_search(path, list_files)
   # list_files = sorted(list_files)
   # print (sess_name + ", #sum files: " + str(len(list_files)))
#extract_feature( list_files, out_file )
#print(list_files[-1])
print(len(list_files))
print(list_files[0])
print(len(labels))
print(labels[0])





2750
IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_F000.wav
2750
neu


***Gettint the labels***

In [ ]:
def get_labels(annot_file, file_name):

    f = open(annot_file, 'r').read()
    f = f.split('\n')
    f = f[2:]

    for data in f:

        if len(data) > 0:
            if data[0] == '[':
                data2 = data.split('\t')

                if data2[1] == file_name:
                    emo = data2[2]
                    vad = data2[3][1:-1].split(', ')
                    return emo, [float(x) for x in vad]

    raise ValueError('Label not found')

In [ ]:
no_rows=len(list_files)
index=0
sprectrogram_shape=[]
docs = []
bookmark=0
extraLabel=0
for everyFile in list_files:
  if(everyFile.split('/')[-1].endswith('.wav')):
    #filename=everyFile.split('/')[-1].strip('.wav')
    i = list_files.index(everyFile)
    lable = labels[i]
    #print('label',lable)
    #if(lable!=-1):
      #sprectrogram_shape.append(audio2spectrogram(everyFile))
    root_path = '/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/'
    everyFile = root_path + everyFile
    spector=audio2spectrogram(everyFile)
    spector=get_3d_spec(spector)
    npimg = np.transpose(spector,(2,0,1))
    input_tensor=torch.tensor(npimg)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
      #X, sample_rate = librosa.load(everyFile, res_type='kaiser_fast',sr=22050*2)
      #sample_rate = np.array(sample_rate)
      #mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
      #feature = mfccs
    docs.append({
      'fileName':everyFile.split('/')[-1].strip('.wav'),
         #'feature_mfcc':feature,
      'sprectrome':input_batch,
       'label':lable
              })
    index+=1
    print('index',index)
  else:
    extraLabel=extraLabel+1
    #print('extraLabel',extraLabel)


***TestAlexNet input***

In [ ]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((12, 12))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        print('features',x.shape)

        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x
def alexnet(pretrained=False, progress=True, **kwargs):
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

***MOdified AlexNet***

In [ ]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ModifiedAlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        #print('features',x.shape)
        x=torch.flatten(x, start_dim=2)#a1,a2,a3......al{a of dim c}
        x=torch.sum(x, dim=2)#a1*alpha1+a2*alpha2+.......+al*alphal
        #print(x.shape)
        x=self.classifier(x)
        #print('classifier',x)
        #x=self.softmax(x)
        #print('softmax',x)
        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x

def modifiedAlexNet(pretrained=False, progress=True, **kwargs):
    model_modified = ModifiedAlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model_modified.load_state_dict(state_dict)
    return model_modified

***create the Modified model instance and initiliaze with the pretraine d model***

In [ ]:
original_model=alexnet(pretrained=True)
original_dict = original_model.state_dict()
modifiedAlexNet=modifiedAlexNet(pretrained=False)
modified_model_dict = modifiedAlexNet.state_dict()
pretrained_modified_model_dict = {k: v for k, v in original_dict.items() if k in modified_model_dict}
modifiedAlexNet.to('cuda')

***Input code to AlexNet with Audio Files***

In [ ]:
for i in range(len(list_files)):
  root_path = '/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/'
  list_files[i] = root_path + list_files[i]

print(list_files[0])


/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_F000.wav


In [ ]:
x=audio2spectrogram(list_files[40])
x=get_3d_spec(x)
npimg = np.transpose(x,(2,0,1))
input_tensor=torch.tensor(npimg)

input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    modifiedAlexNet.to('cuda')
with torch.no_grad():
    output = modifiedAlexNet(input_batch)
    #output.squeeze().shape
    #output=torch.flatten(output, start_dim=2)
    #print(output.shape)
    #output=torch.sum(output, dim=2)
    print(output)

tensor([[-9.3002, -3.8607, -0.9837,  2.5476, 12.3553]])


***Inputdata schuffling and deviding the data***

In [ ]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  2475
no of items for test  275


***Plot Training loss and Accuracy***

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./

***Model Parameter***

In [ ]:
for name, param in modifiedAlexNet.named_parameters():
      if(param.requires_grad):
        print(name)
      else:
        print('no grad',name)

features.0.weight
features.0.bias
features.3.weight
features.3.bias
features.6.weight
features.6.bias
features.8.weight
features.8.bias
features.10.weight
features.10.bias
classifier.1.weight
classifier.1.bias


***optimizer***

In [ ]:
import torch.optim as optim
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(modifiedAlexNet.parameters(),
                  lr =  2e-4,
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

NUM_EPOCHS=16

writer = SummaryWriter(log_dir='/content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


***Training Loop***

In [ ]:
print(train_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
total_steps = 1


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(NUM_EPOCHS):
  modifiedAlexNet.train()
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    label1=torch.tensor([label1])
    sprectrome=every_trainlist['sprectrome']
    if(sprectrome.shape[2]>65):
      optimizer.zero_grad()
      sprectrome = sprectrome.to('cuda')
      label1=label1.to('cuda')
      modifiedAlexNet.zero_grad()
      output = modifiedAlexNet(sprectrome)
      #print('softmax output ',output)
      loss = criterion(output, label1)
      #print('label1',label1)
      #print('loss',loss.item())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(modifiedAlexNet.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
      _, preds = torch.max(output, 1)
      accuracy = torch.sum(preds == label1)
      #print('accuracy.item()',accuracy.item())
      #print('preds',preds)
      if total_steps % 10 == 0:
        with torch.no_grad():
          _, preds = torch.max(output, 1)
          accuracy = torch.sum(preds == label1)
          #print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'.format(epoch + 1, total_steps, loss.item(), accuracy.item()))
          tbwriter.add_scalar('loss', loss.item(), total_steps)
          tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)
      total_steps+=1

ValueError: too many dimensions 'str'

***save and load the model***

In [ ]:
torch.save(modifiedAlexNet, '/content/drive/My Drive/savedModel/model_audio_new_opt.pt')
model=torch.load('/content/drive/My Drive/savedModel/model_audio_new_opt.pt')
model.eval()
model.to('cpu')

***testing lopp***

In [ ]:
y_actu=[]
y_pred=[]
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    sprectrome=every_test_list['sprectrome']
    with torch.no_grad():
      if(sprectrome.shape[2]>65):
        #sprectrome = sprectrome.to('cuda')
        #label1=label1.to('cuda')
        output = model(sprectrome)
        _, preds = torch.max(output, 1)
        y_actu.append(label1.numpy()[0])
        y_pred.append(preds.numpy()[0])

***confusionMatrix***

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)